In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [3]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = df.isnull().sum()/len(df) * 100
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(columns={0:'欠損率', 1:'％'})
    return kesson_table_ren_columns

In [4]:
kesson_table(train)

,欠損率,％
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,19.865320
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [5]:
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Embarked'] = train['Embarked'].fillna('S')

In [6]:
train['Sex'][train['Sex']=='male'] = 0
train['Sex'][train['Sex']=='female'] = 1
train['Embarked'][train['Embarked']=='S'] = 0
train['Embarked'][train['Embarked']=='C'] = 1
train['Embarked'][train['Embarked']=='Q'] = 2

/Users/kenta/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/kenta/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kenta/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel pac

In [7]:
kesson_table(train)

,欠損率,％
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [8]:
kesson_table(test)

,欠損率,％
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,86,20.574163
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,1,0.239234
Cabin,327,78.229665


In [9]:
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [10]:
test['Sex'][test['Sex']=='male'] = 0
test['Sex'][test['Sex']=='female'] = 0
test['Embarked'][test['Embarked']=='S'] = 0
test['Embarked'][test['Embarked']=='C'] = 1
test['Embarked'][test['Embarked']=='Q'] = 2

/Users/kenta/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/kenta/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kenta/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel pac

In [11]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

In [12]:
from sklearn import ensemble

In [13]:
target = train['Survived'].values
train_features = train[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked']].values

In [14]:
n_estimators = 55
random_state = 9
forest = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=random_state)
forest = forest.fit(train_features, target)

In [15]:
# モデルの精度の確認

from sklearn.metrics import accuracy_score
predict_train = forest.predict(train_features)
accuracy_score(predict_train, target)

0.8664421997755332

In [16]:
test_features = test[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked']].values

In [17]:
prediction = forest.predict(test_features)

In [18]:
prediction.shape

(418,)

In [19]:
solution = pd.read_csv('../Data/test.csv')
solution['Survived'] = prediction
solution[['PassengerId', 'Survived']].to_csv('solution_3.csv', index=False)